In [1]:
import unicodecsv # Import csvreader module
from datetime import datetime
from collections import defaultdict
import numpy as np

### Define a function for reading .csv files

In [2]:
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

### Read all three .csv files into tables

In [3]:
enrollments = read_csv('../../../../Data/enrollments.csv')
daily_engagement = read_csv('../../../../Data/daily_engagement.csv')
project_submissions = read_csv('../../../../Data/project_submissions.csv')

For each of these three tables, find the number of rows in the table and
the number of unique students in the table. To find the number of unique
students, you might want to create a set of the account keys in each table.

In [4]:
enrollment_num_rows = len(enrollments) 
unique_enrolled_students = set()
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])
enrollment_num_unique_students = len(unique_enrolled_students)

engagement_num_rows = len(daily_engagement)
unique_engagement_students = set()
for engagement_record in daily_engagement:
    unique_engagement_students.add(engagement_record['acct'])
engagement_num_unique_students = len(unique_engagement_students)

submission_num_rows = len(project_submissions)
unique_project_submitters = set()
for submission in project_submissions:
    unique_project_submitters.add(submission['account_key'])
submission_num_unique_students = len(unique_project_submitters)

In [5]:
print(enrollment_num_rows)
print(enrollment_num_unique_students)

1640
1302


### Rename the acct column to account key

In [6]:
for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]

### Investigate missing engagement records

In [7]:
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students:
        print (enrollment)
        break

{'account_key': '1219', 'status': 'canceled', 'join_date': '2014-11-12', 'cancel_date': '2014-11-12', 'days_to_cancel': '0', 'is_udacity': 'False', 'is_canceled': 'True'}


This student cancelled within a day of joining, which explains why they aren't in the engagement table

### Checking for more problem records 

In [8]:
num_problem_students = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if (student not in unique_engagement_students and 
            enrollment['join_date'] != enrollment['cancel_date']):
        print (enrollment)
        num_problem_students += 1

num_problem_students

{'account_key': '1304', 'status': 'canceled', 'join_date': '2015-01-10', 'cancel_date': '2015-03-10', 'days_to_cancel': '59', 'is_udacity': 'True', 'is_canceled': 'True'}
{'account_key': '1304', 'status': 'canceled', 'join_date': '2015-03-10', 'cancel_date': '2015-06-17', 'days_to_cancel': '99', 'is_udacity': 'True', 'is_canceled': 'True'}
{'account_key': '1101', 'status': 'current', 'join_date': '2015-02-25', 'cancel_date': '', 'days_to_cancel': '', 'is_udacity': 'True', 'is_canceled': 'False'}


3

Three problem records, these all have is_udacity = True, because they are not enrolled but are udacity records


#### Removing Udacity accounts

In [9]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
       udacity_test_accounts.add(enrollment['account_key'])
       
len(udacity_test_accounts) 

1302

In [10]:
# def remove_udacity_accounts(data):
    # non_udacity_data = []
    # for data_point in data:
        # if data_point['account_key'] not in udacity_test_accounts:
            # non_udacity_data.append(data_point)
    # return non_udacity_data

In [11]:
# non_udacity_enrollments = remove_udacity_accounts(enrollments)
# non_udacity_engagement = remove_udacity_accounts(daily_engagement)
# non_udacity_submissions = remove_udacity_accounts(project_submissions)

# print(len(non_udacity_enrollments))
# print(len(non_udacity_engagement))
# print(len(non_udacity_submissions))

### Refining the question


In [12]:
paid_students = {}
for enrollment in enrollments:
    # Ensure 'days_to_cancel' is not an empty string before converting to an integer
    if (not enrollment['is_canceled'] or
            (enrollment['days_to_cancel'] and int(enrollment['days_to_cancel']) > 7)):
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        if (account_key not in paid_students or
                enrollment_date > paid_students[account_key]):
            paid_students[account_key] = enrollment_date

len(paid_students)

448

### Getting Data from First Week

In [13]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days >= 0 and time_delta.days < 7
    
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data
    
paid_enrollments = remove_free_trial_cancels(enrollments)
paid_engagement = remove_free_trial_cancels(daily_engagement)
paid_submissions = remove_free_trial_cancels(project_submissions)
    
print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))
    
paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']
        
    if within_one_week(datetime.strptime(join_date, '%Y-%m-%d'), datetime.strptime(engagement_record_date, '%Y-%m-%d')):
         paid_engagement_in_first_week.append(engagement_record)
    
len(paid_engagement_in_first_week)


734
56254
1118


3129

In [14]:
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account['account_key'].append(engagement_record)

In [15]:
total_minutes_by_account = {}

for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += float(engagement_record['total_minutes_visited'])
    total_minutes_by_account['account_key'] = total_minutes

In [16]:
# total_minutes = total_minutes_by_account.values()

print(np.mean(total_minutes))
print(np.std(total_minutes))
print(np.min(total_minutes))
print(np.max(total_minutes))

109343.32818166321
0.0
109343.32818166321
109343.32818166321


In [17]:
student_with_max_minutes = None
max_minutes = 0

for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student
        
max_minutes

for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] == student_with_max_minutes:
        print(engagement_record)

In [18]:
max(total_minutes_by_account.items(), key=lambda pair: pair[1])

('account_key', 109343.32818166321)

### Lessons completed first week


In [23]:
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

engagement_by_account = group_data(paid_engagement_in_first_week,
                                   'account_key')

def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data


import numpy as np

def describe_data(data):
    print('Mean:', np.mean(data))
    print('Standard deviation:', np.std(data))
    print('Minimum:', np.min(data))
    print('Maximum:', np.max(data))
